## Analyse des bases de données 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df1 = pd.read_csv('df1.csv',delimiter=',')
df2 = pd.read_csv('df2.csv',delimiter=',')
print("la taille de la dataset donnant la consommation en fonction de la température est ", df2.shape[0]) # sans les données à compléter
print("la taille de la dataset complète donnant la consommation en fonction de la température est ", df2.dropna().shape[0]) # avec les données à compléter
print("la taille de la dataset donnant la consommation en fonction de la température est ", df1.shape[0]) 
print("la taille de la dataset complète donnant la consommation en fonction de la température est ", df1.dropna().shape[0]) 
print('la différence entre les différentes datasets complète et non complète est \n', pd.concat([df2,df2.dropna()]).drop_duplicates(keep=False).head())
print(df1.columns)


On conclu qu'il n'y a pas de données erronés, la différence des datasets trouvé est juste la partie du fichier csv qu'on doit compléter. 

In [ ]:
f, axes = plt.subplots(3, 3, figsize=(20,20))
sns.boxplot(df2['mean'],ax=axes[0,0])
sns.boxplot(df2['temp'],ax=axes[1,1])
sns.boxplot(df1['rain'],ax=axes[2,2])
sns.boxplot(df1['sun'],ax=axes[1,0])
sns.boxplot(df1['rhum'],ax=axes[0,1])



Exploration des propriétés temporelles des deux datasets


In [ ]:
print(df2['time'].max())
print(df1['date'].max())
# La dataset 2 a pour décalage une demie heure, La dataset 1 a pour décalage une demi heure.
print(df1['date'])

## Rassemblement des deux datasets


In [ ]:
clean = df2
clean['time'] = pd.to_datetime(clean['time'])
df2_= clean[clean['time'].dt.minute == 00]
df2_=df2_.dropna()
df1_=df1[(df1['date'] >='2009-07-15 ') & (df1['date'] <= '2010-12-15')]
df1_.rename(columns={'date':'time'}, inplace=True)
df1_['time'] = pd.to_datetime(df1_['time'])
df = pd.merge(left=df1_,right=df2_,on='time' )
df=df.drop("temp_x", axis='columns')
df.rename(columns={'temp_y':'temp'}, inplace=True)

## Visualisation et calcul des différentes corrélation avec la consommation électrique

In [ ]:
columns = df.columns
for name in columns : 
    try : 
        sns.pairplot(df,x_vars=[name], y_vars='mean', height=4, aspect=3, kind='scatter')
    except: 
        print(name)
kerr


### Remarque: 
Nous remarquons que l'humidité 'rhum', la hauteur des nuages 'clht' , la vapeur d'eau 'vappr' , la température 'temp' et la pluie 'rain' ont une corrélation importante avec la consommation électrique. Cependant on va calculer ces corrélations. 

### Indices de corrélation


In [ ]:
print([[name_column for name_column in df.columns if name_column]])
columns = ['time','temp','mean','vappr','ind','ind.1','rain','rhum','msl','wdsp','wddir','ww','w','sun','clht','clamt',]
L= ['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
       'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir', 'ww', 'w',
       'sun', 'vis', 'clht', 'clamt'] - columns
columns = ['time','temp','mean','vappr','ind','ind.1','rain','rhum','msl','wdsp','wddir','ww','w','sun','clht','clamt',]
covariance_matrix = df[['temp','mean','vappr','ind','ind.1','rain','rhum','msl','wdsp','wddir','ww','w','sun','clht','clamt',]].corr() #ind.3,ind.4 : valeurs nulles, 
covariance_result = covariance_matrix.iloc[1]
noms = covariance_result.keys
values = covariance_result.values
values = [abs(value) for value in values] 

d = {('consommation',columns[i]) : values[i] for i in range(len(values)) if columns[i] != 'mean'}
d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse =True)}


[['time', 'ind', 'rain', 'ind.1', 'ind.2', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir', 'ww', 'w', 'sun', 'vis', 'clht', 'clamt', 'mean', 'temp']]


TypeError: unsupported operand type(s) for -: 'list' and 'list'

### Commentaire
On a bien que le niveau de vapeur d'eau, la température et l'humidité sont les plus corrélés avec la consommation électrique 

## Remarque 
On remarque bien que les données tout au long de l'année ne suffisent pas, on va maintenant s'intéresser aux mois individuelles et voir la variation mensuelle, quotidienne et horaire de la consommation électrique

In [ ]:
from datetime import datetime

def dataset_mois(année,mois):# retourne les valeurs des grandeurs pour le mois en argument
    return df[(df['time'].dt.year == année) & (df['time'].dt.month ==mois)]
def dataset_semaine(année,mois):#retourne les valeurs des grandeurs hebdomadaires 
    df_m = df[(df['time'].dt.year == année) & (df['time'].dt.month == mois)]
    dates = df_m['time'].to_list()
    df_m['time'] = df_m['time'].dt.strftime('%d %m %Y')
    df_mm = df_m['time'].drop_duplicates()
    dates = df_mm.to_list()
    i=0
    for k in range(len(dates)):
        if pd.to_datetime(dates[k]).weekday() == 0:
            i=k
            break
    new_dates = dates[i:]
    n=len(new_dates)
    new_dates = new_dates[:int(np.floor(n/7) * 7)]
    days = [int(s[:2]) for s in new_dates ]
    Df = df[(df['time'].dt.year == année) & (df['time'].dt.month == mois)]
    df_mmm = Df[Df['time'].dt.day.isin(days)]
    return df_mmm
print(dataset_semaine(2010,1).head())
def dataset_journée(année,mois,journée):
    return df[(df['time'].dt.year == année) & (df['time'].dt.month == mois) & (df['time'].dt.day == journée)]
print(dataset_journée(2009,12,30).head())


Corrélation entre consommation électrique et les autres variables explicatives pour le mois de décembre 2009

In [ ]:
columns = dataset_mois(2009,12).columns
for name in columns : 
    try : 
        sns.pairplot(dataset_mois(2009,12),x_vars=[name], y_vars='mean', height=4, aspect=3, kind='scatter')
    except: 
        print(name)

In [ ]:
df_mois=dataset_mois(2009,12)
columns = ['temp','mean','vappr','ind','ind.1','rain','rhum','msl','wdsp','wddir','ww','w','sun','clht','clamt',]
covariance_matrix = df_mois[['temp','mean','vappr','ind','ind.1','rain','rhum','msl','wdsp','wddir','ww','w','sun','clht','clamt',]].corr() #ind.3,ind.4 : valeurs nulles, 
covariance_result = covariance_matrix.iloc[1]
noms = covariance_result.keys
values = covariance_result.values
values = [abs(value) for value in values] 

d = {('consommation',columns[i]) : values[i] for i in range(len(values)) if columns[i] != 'mean'}
d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse =True)}
print(d)

In [ ]:
columns = dataset_journée(2009,2,1).columns
for name in columns : 
    try : 
        sns.pairplot(dataset_journée(2010,8,2),x_vars=[name], y_vars='mean', height=4, aspect=3, kind='scatter')
    except: 
        print(name)

In [ ]:
plt.plot(dataset_semaine(2009,10).iloc[:7*24]['time'].to_list(), dataset_semaine(2009,10).iloc[:7*24]['mean'].to_list(), '-')
plt.show()